In [86]:
import base64
import json
from google.oauth2.credentials import Credentials
import base64
from google_auth_oauthlib.flow import InstalledAppFlow
import os
from email.mime.text import MIMEText
from googleapiclient.discovery import build
from email import message_from_bytes
from email.header import decode_header

In [87]:
SCOPES = ["https://mail.google.com/"]

In [88]:
def build_service():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
        # Save the credentials for next run
        with open("token.json", "w") as token_file:
            token_file.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

In [89]:
service = build_service()

In [97]:
q=[]
dic = {
    "start date":'10-10-2025',
    "end date":'10-10-2025',
    "name":"null",
}

for key,val in dic.items():
    print(key,val)
    if val != "null":
        q.append(f"{key}:{val} ")

print(q)

print("".join(q))

start date 10-10-2025
end date 10-10-2025
name null
['start date:10-10-2025 ', 'end date:10-10-2025 ']
start date:10-10-2025 end date:10-10-2025 


In [126]:
filters = """```json
{
    "after": "2025/10/09",
    "before": "2025/10/10",
    "from": "Sameer khan",
    "limit": null,
    "subject": null
}
```"""

In [127]:
import json
import re
clean_json = re.sub(r"^```json\s*|\s*```$", "", filters.strip())
filters = json.loads(clean_json)
print(filters)


{'after': '2025/10/09', 'before': '2025/10/10', 'from': 'Sameer khan', 'limit': None, 'subject': None}


In [128]:
q=[]
for key,val in filters.items():
    print(key,val)
    if val != None:
        q.append(f"{key}:{val} ")

print(q)
query = "".join(q)
print(query)

after 2025/10/09
before 2025/10/10
from Sameer khan
limit None
subject None
['after:2025/10/09 ', 'before:2025/10/10 ', 'from:Sameer khan ']
after:2025/10/09 before:2025/10/10 from:Sameer khan 


In [129]:
#query = "after:2025/10/09 before:2025/10/10"
service.users().messages().list(userId="me",q=query, maxResults=5).execute()


{'messages': [{'id': '199c8404e2ec7fe0', 'threadId': '199c8404e2ec7fe0'}],
 'resultSizeEstimate': 1}

In [130]:
service.users().messages().get(userId="me", id='199c8404e2ec7fe0', format="full").execute()


{'id': '199c8404e2ec7fe0',
 'threadId': '199c8404e2ec7fe0',
 'labelIds': ['UNREAD', 'IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'],
 'snippet': 'hi there this is my first testing of email agent on 09-0ct',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'youthofvaranasi1999@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a17:505:a82:b0:1d4e:b5b5:c92f with SMTP id ky2csp1456423njb;        Thu, 9 Oct 2025 02:14:41 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:6122:3bc3:b0:554:b4e8:a99a with SMTP id 71dfb90a1353d-554b8ba0b55mr2907975e0c.9.1760001281756;        Thu, 09 Oct 2025 02:14:41 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1760001281; cv=none;        d=google.com; s=arc-20240605;        b=cTdvNvef1y0lC99lktKLJbxFptG7m0eMV6WmXtdGayz/tcQS7hvuS/AeKQmRDPeui3         ViogS3vfWV7jrMEzAHifl3GAzyx0TLmxviJyY1vgNQoGve/bIKi+Gpmx+VRjI84lRay6         w

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(service.users().messages().list(userId="me", maxResults=5).execute()
)

In [9]:
df

,messages,nextPageToken,resultSizeEstimate
0,"{'id': '199bf415b17a8937', 'threadId': '199bf4...",00502821322337245476,201
1,"{'id': '199bf3f18480ffc3', 'threadId': '199bf3...",00502821322337245476,201
2,"{'id': '199bef05a57c79fc', 'threadId': '199bef...",00502821322337245476,201
3,"{'id': '199bee89a163c422', 'threadId': '199bed...",00502821322337245476,201
4,"{'id': '199bedc9499927bd', 'threadId': '199bed...",00502821322337245476,201


In [10]:
df['messages'].head()

0    {'id': '199bf415b17a8937', 'threadId': '199bf4...
1    {'id': '199bf3f18480ffc3', 'threadId': '199bf3...
2    {'id': '199bef05a57c79fc', 'threadId': '199bef...
3    {'id': '199bee89a163c422', 'threadId': '199bed...
4    {'id': '199bedc9499927bd', 'threadId': '199bed...
Name: messages, dtype: object

In [13]:
df.to_csv('data.csv')

In [14]:
sample_sent = service.users().messages().get(userId="me", id='199bf415b17a8937', format="full").execute()
sample_rec = service.users().messages().get(userId="me", id='199bef05a57c79fc', format="full").execute()

In [15]:
sample_sent

{'id': '199bf415b17a8937',
 'threadId': '199bf415b17a8937',
 'labelIds': ['SENT'],
 'snippet': 'Hello! This is a test email sent via Gmail API using Python.',
 'payload': {'partId': '',
  'mimeType': 'text/plain',
  'filename': '',
  'headers': [{'name': 'Received',
    'value': 'from 259311140045 named unknown by gmailapi.google.com with HTTPREST; Tue, 7 Oct 2025 15:19:15 +0000'},
   {'name': 'Received',
    'value': 'from 259311140045 named unknown by gmailapi.google.com with HTTPREST; Tue, 7 Oct 2025 15:19:15 +0000'},
   {'name': 'Content-Type', 'value': 'text/plain; charset="us-ascii"'},
   {'name': 'MIME-Version', 'value': '1.0'},
   {'name': 'Content-Transfer-Encoding', 'value': '7bit'},
   {'name': 'to', 'value': 'sameerkhan1ssk1@gmail.com'},
   {'name': 'From', 'value': 'youthofvaranasi1999@gmail.com'},
   {'name': 'subject', 'value': 'Test Email from Gmail API this is sameer'},
   {'name': 'Date', 'value': 'Tue, 7 Oct 2025 15:19:15 +0000'},
   {'name': 'Message-Id',
    'value

In [34]:
results = service.users().messages().list(
        userId="me",
        q=query,
        maxResults=100
    ).execute()

NameError: name 'query' is not defined

In [16]:
sample_rec

{'id': '199bef05a57c79fc',
 'threadId': '199bef05a57c79fc',
 'labelIds': ['CATEGORY_PERSONAL', 'INBOX'],
 'snippet': 'hi there i am sameer khan, and i am working on email agent. thankyou sameer khan',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'youthofvaranasi1999@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a17:505:a82:b0:1d4e:b5b5:c92f with SMTP id ky2csp254897njb;        Tue, 7 Oct 2025 06:50:47 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:6102:161e:b0:5a2:8581:e76a with SMTP id ada2fe7eead31-5d41d12d12bmr5480516137.31.1759845046913;        Tue, 07 Oct 2025 06:50:46 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1759845046; cv=none;        d=google.com; s=arc-20240605;        b=brP7OkjiaYHSbeAiXKY/PN9zdZn/9AYHTbH6V0/I/ciIuwC7mMKNGOZGl1oQZsACzv         FgepmB2DDHRSm6BN7euMVkXKeQamhUxCA7l4Z1rYtQgCyPewtTF7HER4x1rFh/96QP/4         7e

In [27]:
pd.DataFrame(sample_rec['payload']['headers']).loc[lambda x: x['name'] == 'From']

,name,value
17,From,Sameer Khan <sameerkhan1ssk1@gmail.com>


In [25]:
sample_rec['snippet']

'hi there i am sameer khan, and i am working on email agent. thankyou sameer khan'

In [22]:
pd.DataFrame(sample_rec['payload']['headers'])

,name,value
0,Delivered-To,youthofvaranasi1999@gmail.com
1,Received,by 2002:a17:505:a82:b0:1d4e:b5b5:c92f with SMT...
2,X-Received,by 2002:a05:6102:161e:b0:5a2:8581:e76a with SM...
3,ARC-Seal,i=1; a=rsa-sha256; t=1759845046; cv=none; ...
4,ARC-Message-Signature,i=1; a=rsa-sha256; c=relaxed/relaxed; d=google...
5,ARC-Authentication-Results,i=1; mx.google.com; dkim=pass header.i=@...
6,Return-Path,<sameerkhan1ssk1@gmail.com>
7,Received,from mail-sor-f41.google.com (mail-sor-f41.goo...
8,Received-SPF,pass (google.com: domain of sameerkhan1ssk1@gm...
9,Authentication-Results,mx.google.com; dkim=pass header.i=@gmail...
